# Lagrangian Neural Networks

In this tutorial we take a look at **Lagrangian Nets** (LNNs) first proposed in [Lutter M., et al, 2019](https://arxiv.org/abs/1907.04490) and generalized by [Cranmer M., et al., 2020](https://arxiv.org/abs/2003.04630). Together with HNNs [Greydanus S., et al., 2019](https://arxiv.org/abs/1906.01563), LNNs represent the latest learning paradigm to discover symmetries and conservation laws from data.

Let $\mathcal{Q}\subset\mathbb{R}^n$ be a smooth manifold anf let $q\in\mathcal{Q}$ be a vector of generalized coordinates. The Lagrangian function $\mathcal{L}:\mathcal{TQ}\rightarrow\mathbb{R}$ is defined on the tangent bundle $\mathcal{TQ}$ of the configuration manifold $\mathcal{Q}$ (if $\mathcal{Q}=\mathbb{R}^n$, then $\mathcal{TQ}$ is diffeomorphic to $\mathbb{R}^{2n}$), i.e. the Lagrangian is a function of the configurations $q$ and their velocities $\dot q$. 

Derived from the calculus of variations, the Euler-Lagrange equations of motions can be generally explicitly written as a second--order ordinary differential equation:

$$
    \ddot q = \left(\nabla_{\dot q} \nabla_{\dot q}^\top \mathcal{L}\right)^{-1}\left[\nabla_q\mathcal{L} - \left(\nabla_q\nabla^\top_{\dot q}\mathcal L\right)\dot q\right]
$$

**Lagrangian Neural Networks** try to mimick Euler-Lagrange equations by learning from data a Lagrangian $\mathcal{L}_\theta$ parametriezed by a *neural network* (with parameters $\theta$). 

When a net of tuples $\{(q_k,\dot q_k, \ddot q_k)\}_{k=1,\dots,K}$ generated by some conservative dynamical process is available, LNNs are trained to learn the Lagrangian from the data by minimizing, e.g. the *MSE loss*:
$$
    \min_{\theta}\frac{1}{K}\sum_{k=1}^K\left\|\ddot q_k - \left(\nabla_{\dot q} \nabla_{\dot q}^\top \mathcal{L}_\theta(q_k, \dot q_k)\right)^{-1}\left[\nabla_q\mathcal{L}_\theta(q_k, \dot q_k) - \left(\nabla_q\nabla^\top_{\dot q}\mathcal L_\theta(q_k, \dot q_k)\right)\dot q_k\right]\right\|_2^2
$$


We hereby showcase the torchdyn implementation of LNNs.

In [1]:
import sys
sys.path.append('../')
from torchdyn.models import *; from torchdyn.datasets import *
from torchdyn import *

The vector field of an LNN, to be passed to a `NeuralDE` can be defined as:

In [2]:
from torch.autograd.functional import jacobian, hessian ; from functools import partial

class LNN(nn.Module):
    def __init__(self, net):
        super().__init__()
        self.net = net
    def forward(self, x):
        self.n = n = x.shape[1]//2 ; bs = x.shape[0]   
        x = torch.autograd.Variable(x, requires_grad=True)
        qqd_batch = tuple(x[i, :] for i in range(bs))
        jac = tuple(map(partial(jacobian, self._lagrangian, create_graph=True), qqd_batch))
        hess = tuple(map(partial(hessian, self._lagrangian, create_graph=True), qqd_batch))
        qdd_batch = tuple(map(self._qdd, zip(jac, hess, qqd_batch)))
        qd, qdd = x[:, n:], torch.cat([qdd[None] for qdd in qdd_batch])
        return torch.cat([qd, qdd], 1)
    
    def _lagrangian(self, qqd):
        return self.net(qqd).sum()
    
    def _qdd(self, inp):
        n = self.n ; jac, hess, qqd = inp
        return hess[n:, n:].pinverse()@(jac[:n] - hess[n:, :n]@qqd[n:])

We consider a 1D pendulum with torsional spring

$$
    m\ddot x - kx - mgl\sin(x) = 0
$$

Following the LNN paper from Cranmer et al., we generate random values of $x,\dot x$ and we fit (in a static manner) the corresponding $\ddot q$

## "Static" Training

In [3]:
import torch.utils.data as data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

m, k, l, g = 1, 1, 1, 9.81
X = torch.randn(1000,2).to(device)
Xdd = -k*X[:,0]/m - m*g*l*torch.sin(X[:,0])

train = data.TensorDataset(X, Xdd)
trainloader = data.DataLoader(train, batch_size=len(X)//10, shuffle=False)

In [4]:
import pytorch_lightning as pl

class Learner(pl.LightningModule):
    def __init__(self, model:nn.Module):
        super().__init__()
        self.model = model
    
    def forward(self, x):
        return self.model.defunc(0, x)
    
    def loss(self, y_hat, y):
        return ((y - y_hat[:,1])**2).mean()
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model.defunc(0, x) # static training: we do not solve the ODE 
        loss = self.loss(y_hat, y)
        return {'loss': loss}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.0001)

    def train_dataloader(self):
        return trainloader

In [5]:
net = LNN(nn.Sequential(
            nn.Linear(2,64),
            nn.Softplus(),
            nn.Linear(64,64),
            nn.Softplus(),
            nn.Linear(64,1))
         ).to(device)

model = NeuralDE(func=net, solver='dopri5').to(device)

In [ ]:
learn = Learner(model)
trainer = pl.Trainer(min_epochs=500, max_epochs=1000)
trainer.fit(learn)

INFO:lightning:
  | Name                 | Type       | Params
------------------------------------------------
0 | model                | NeuralDE   | 4 K   
1 | model.defunc         | DEFunc     | 4 K   
2 | model.defunc.m       | LNN        | 4 K   
3 | model.defunc.m.net   | Sequential | 4 K   
4 | model.defunc.m.net.0 | Linear     | 192   
5 | model.defunc.m.net.1 | Softplus   | 0     
6 | model.defunc.m.net.2 | Linear     | 4 K   
7 | model.defunc.m.net.3 | Softplus   | 0     
8 | model.defunc.m.net.4 | Linear     | 65    


C:\Users\stefano\Anaconda3\lib\site-packages\pytorch_lightning\utilities\warnings.py:18: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)


In [ ]:
# Evaluate the training results
import time
t = time.time()
model.nfe = 0
X0 = torch.Tensor(256, 2).uniform_(-1,1).to(device)
s_span = torch.linspace(0, 1, 100)
traj = model.trajectory(X0, s_span).cpu().detach()
T = time.time() - t
print(f"NFE: {model.nfe}\ninference time: {T}\navg time per function evaluation: {T/model.nfe}")
# the inference time to compute a full trajectory is currently still quite high and scales with the batch. We are aware of this issue and we will release an accellerated version soon 

In [ ]:
# Plot the HHN's trajectories with random ICs
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(111)
color = ['orange', 'blue']
for i in range(len(X0)):
    ax.plot(traj[:,i,0], traj[:,i,1], color='blue', alpha=0.1);
#ax.plot(X[:,0].cpu(),X[:,1].cpu(), color='red')
ax.set_xlim([-1,1])
ax.set_ylim([-4,4])
ax.set_xlabel(r"$q$")
ax.set_ylabel(r"$p$")
ax.set_title("HHN's trajectories & training data")

In [ ]:
n_grid =  50
x = torch.linspace(-3,3,n_grid)
Q, P = torch.meshgrid(x,x)
H, U, V = torch.zeros(Q.shape), torch.zeros(Q.shape), torch.zeros(Q.shape)
for i in range(n_grid):
    for j in range(n_grid):
        x = torch.cat([Q[i,j].reshape(1,1),P[i,j].reshape(1,1)],1).to(device)
        H[i,j] = model.defunc.m.net(x).detach().cpu()
        O = model.defunc(0,x).detach().cpu()
        U[i,j], V[i,j] = O[0,0], O[0,1]
        
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(111)
ax.contourf(Q,P,H,100,cmap='RdYlBu')
ax.streamplot(Q.T.numpy(),P.T.numpy(),U.T.numpy(),V.T.numpy(), color='black')
ax.set_xlim([Q.min(),Q.max()])
ax.set_ylim([P.min(),P.max()])
ax.set_xlabel(r"$q$")
ax.set_ylabel(r"$p$")
ax.set_title("Learned Lagrangian & Vector Field")